In [58]:
import numpy as np
import statsmodels.api as sm
from reba_optim.reba_assess import RebaAssess
from human_moveit_config.human_model import HumanModel
import csv
from scipy.optimize import minimize

In [22]:
def generate_random_pose(database, A_table, B_table, C_table):
    side = ['right', 'left']
    X = []
    # random pose for A group
    i_n0 = np.random.randint(4)
    i_n1 = np.random.randint(5)
    i_n2 = np.random.randint(4)
    X.append(database['neck_0']['joints'][i_n0])
    X.append(database['neck_1']['joints'][i_n1])
    X.append(database['neck_2']['joints'][i_n2])
    n_val = database['neck_1']['values'][i_n1]
    if database['neck_0']['values'][i_n0] == 1 or database['neck_2']['values'][i_n2] == 1:
        n_val += 1

    i_s0 = np.random.randint(4)
    i_s1 = np.random.randint(5)
    i_s2 = np.random.randint(4)
    X.append(database['spine_0']['joints'][i_s0])
    X.append(database['spine_1']['joints'][i_s1])
    X.append(database['spine_2']['joints'][i_s2])
    s_val = database['spine_1']['values'][i_s1]
    if database['spine_0']['values'][i_s0] == 1 or database['spine_2']['values'][i_s2] == 1:
        s_val += 1
    
    k_vals = []
    for i in range(2):
        i_l0 = np.random.randint(3)
        X.append(database['knee']['joints'][i_l0])
        k_vals.append(database['knee']['values'][i_l0])
    k_val = max(k_vals)
    # get value from table
    A_val = A_table[n_val - 1, s_val - 1, k_val - 1]
    
    # random pose for B group
    sh_vals = []
    for i in range(2):
        i_sh0 = np.random.randint(4)
        i_sh1 = np.random.randint(6)
        X.append(database[side[i] + '_shoulder_0']['joints'][i_sh0])
        X.append(database[side[i] + '_shoulder_1']['joints'][i_sh1])
        sh_vals.append(database[side[i] + '_shoulder_1']['values'][i_sh1])
        if database[side[i] + '_shoulder_0']['values'][i_sh0] == 1:
            sh_vals[-1] += 1
    sh_val = max(sh_vals)

    e_vals = []
    for i in range(2):
        i_e0 = np.random.randint(4)
        X.append(database[side[i] + '_elbow_0']['joints'][i_e0])
        e_vals.append(database[side[i] + '_elbow_0']['values'][i_e0])
    e_val = max(e_vals)

    w_vals = []
    for i in range(2):
        i_w0 = np.random.randint(3)
        i_w1 = np.random.randint(4)
        X.append(database['wrist_0']['joints'][i_w0])
        X.append(database['wrist_1']['joints'][i_w1])
        w_vals.append(database['wrist_1']['values'][i_w1])
        if database['wrist_0']['values'][i_w0] == 1:
            w_vals[-1] += 1
    w_val = max(w_vals)
    
    # get value from table
    B_val = B_table[e_val - 1, sh_val - 1, w_val - 1]
    
    # get total value from C table
    C_val = C_table[int(A_val - 1), int(B_val - 1)]

    return X, C_val

In [66]:
def evaluate(weights, X, Y):
    cost = 0
    for i in range(len(X)):
        estimate = 0
        for j in range(len(X[i])):
            poly = 0
            for k in range(4):
                poly += weights[j * 5 + k] * X[i][j]**k
            estimate += weights[j * 5 + 4] * poly
        # check difference with ground truth
        cost += (estimate - Y[i])**2
    return cost

In [3]:
def quality_check(q, weights):
    return model.predict(q)

In [4]:
reba = RebaAssess()
human = HumanModel()

In [23]:
joint_names = ['neck_0', 'neck_1', 'neck_2', 'spine_0', 'spine_1', 'spine_2',
               'right_knee', 'left_knee', 'right_shoulder_0', 'right_shoulder_1',
               'left_shoulder_0', 'left_shoulder_1', 'right_elbow_0', 'left_elbow_0',
               'right_wrist_0', 'right_wrist_1', 'left_wrist_0', 'left_wrist_1']
limits = human.get_joint_limits(joint_names)

In [24]:
rad_factor = np.pi / 180.
rad_convert = lambda x : x * rad_factor

In [34]:
database = {}
# neck
database['neck_0'] = {}
database['neck_0']['joints'] = map(rad_convert, [-45, -15, 15, 45])
database['neck_0']['values'] = [1, 0, 0, 1]
database['neck_1'] = {}
database['neck_1']['joints'] = map(rad_convert, [-45, 0, 10, 20, 45])
database['neck_1']['values'] = [2, 2, 1, 1, 2]
database['neck_2'] = {}
database['neck_2']['joints'] = map(rad_convert, [-45, -15, 15, 45])
database['neck_2']['values'] = [1, 0, 0, 1]
# trunk
database['spine_0'] = {}
database['spine_0']['joints'] = map(rad_convert, [-45, -15, 15, 45])
database['spine_0']['values'] = [1, 0, 0, 1]
database['spine_1'] = {}
database['spine_1']['joints'] = map(rad_convert, [-20, 0, 20, 60, 80])
database['spine_1']['values'] = [2, 1, 2, 3, 4]
database['spine_2'] = {}
database['spine_2']['joints'] = map(rad_convert, [-45, -15, 15, 45])
database['spine_2']['values'] = [1, 0, 0, 1]
# knee 
database['knee'] = {}
database['knee']['joints'] = map(rad_convert, [-80, -60, -30, 0])
database['knee']['values'] = [2, 1, 1, 2]
# shoulder right
database['right_shoulder_0'] = {}
database['right_shoulder_0']['joints'] = map(rad_convert, [-90, 0, 45, 90])
database['right_shoulder_0']['values'] = [1, 1, 0, 0]
database['right_shoulder_1'] = {}
database['right_shoulder_1']['joints'] = map(rad_convert, [-45, -20, 20, 45, 90, 135])
database['right_shoulder_1']['values'] = [2, 1, 1, 2, 3, 4]
# shoulder left
database['left_shoulder_0'] = {}
database['left_shoulder_0']['joints'] = map(rad_convert, [-90, 0, 45, 90])
database['left_shoulder_0']['values'] = [1, 1, 0, 0]
database['left_shoulder_1'] = {}
database['left_shoulder_1']['joints'] = map(rad_convert, [45, 20, -20, -45, -90, -135])
database['left_shoulder_1']['values'] = [2, 1, 1, 2, 3, 4]
# elbow right
database['right_elbow_0'] = {}
database['right_elbow_0']['joints'] = map(rad_convert, [0, 60, 100, 145])
database['right_elbow_0']['values'] = [2, 1, 1, 2]
# elbow left
database['left_elbow_0'] = {}
database['left_elbow_0']['joints'] = map(rad_convert, [0, -60, -100, -145])
database['left_elbow_0']['values'] = [2, 1, 1, 2]
# wrist
database['wrist_0'] = {}
database['wrist_0']['joints'] = map(rad_convert, [-15, 0, 15])
database['wrist_0']['values'] = [1, 0, 1]
database['wrist_1'] = {}
database['wrist_1']['joints'] = map(rad_convert, [-30, -15, 15, 30])
database['wrist_1']['values'] = [2, 1, 1, 2]

In [75]:
X = []
Y = []
for i in range(100000):
    joints, val = generate_random_pose(database, reba.reba_A_table, reba.reba_B_table, reba.reba_C_table)
    X.append(joints)
    Y.append(val)

In [ ]:
valid_X = []
valid_Y = []
for j in range(100):
    joints, val = generate_random_pose(database, reba.reba_A_table, reba.reba_B_table, reba.reba_C_table)
    valid_X.append(joints)
    valid_Y.append(val)

In [76]:
f = open('/tmp/database.csv', 'wt')
try:
    writer = csv.writer(f)
    writer.writerow(['Y'] + joint_names)
    for i in range(len(X)):
        writer.writerow([Y[i]] + X[i])
finally:
    f.close()

In [71]:
init_weights = np.ones(len(X[0]) * 5)

In [72]:
res = minimize(evaluate, init_weights,
               args=(X, Y),
               method='L-BFGS-B')

KeyboardInterrupt: 

In [ ]:
res